# LOAD BALANCING in NGINX

# https://www.upcloud.com/support/how-to-set-up-load-balancing/

<img src="https://www.upcloud.com/support/wp-content/uploads/2015/12/load-balancer-graph-1-1024x522.png">


# Virtual host

* ```mv load-balancer.conf load-balancer.conf.disabled```
* http://test1.com:8080/

# Load balancer

* ```mv default.conf default.conf.disabled```
* http://localhost:8080/webapp/

In [6]:
%%writefile code/docker-compose.yml
version: "2"

services:
  node1:
    image: "node:8"
    user: "node"
    working_dir: /home/node/app
    environment:
      - NODE_ENV=production
    volumes:
      - /home/andrea/Documents/University/Didattica/RC2018/notebooks/LOAD_BALANCING_HTTPS/code/app:/home/node/app
    expose:
      - 8081
    command: "npm start"
  node2:
    image: "node:8"
    user: "node"
    working_dir: /home/node/app
    environment:
      - NODE_ENV=production
    volumes:
      - /home/andrea/Documents/University/Didattica/RC2018/notebooks/LOAD_BALANCING_HTTPS/code/app:/home/node/app
    expose:
      - 8081
    command: "npm start"
  ngix:
    image: "nginx"
    volumes:
    
      - /home/andrea/Documents/University/Didattica/RC2018/notebooks/LOAD_BALANCING_HTTPS/code/var/www:/var/www
      - /home/andrea/Documents/University/Didattica/RC2018/notebooks/LOAD_BALANCING_HTTPS/code/nginx.conf:/etc/nginx/nginx.conf
      - /home/andrea/Documents/University/Didattica/RC2018/notebooks/LOAD_BALANCING_HTTPS/code/conf.d:/etc/nginx/conf.d
    ports:
      - "8080:80"
    expose:
      - 80
#  couchdb:
#    image: couchdb
#    ports:
#      - "5984:5984"
#    expose:
#      - 5984   



Overwriting code/docker-compose.yml


In [7]:
%%writefile code/nginx.conf

user  nginx;
worker_processes  1;

error_log  /var/log/nginx/error.log warn;
pid        /var/run/nginx.pid;


events {
    worker_connections  1024;
}


http {
    include       /etc/nginx/mime.types;
    default_type  application/octet-stream;

    log_format  main  '$remote_addr - $remote_user [$time_local] "$request" '
                      '$status $body_bytes_sent "$http_referer" '
                      '"$http_user_agent" "$http_x_forwarded_for"';

    access_log  /var/log/nginx/access.log  main;

    sendfile        on;
    #tcp_nopush     on;

    keepalive_timeout  65;

    #gzip  on;

    include /etc/nginx/conf.d/*.conf;
    #include /etc/nginx/conf.d/sites-available/*.conf;
    #include /etc/nginx/conf.d/sites-enabled/*.conf;
}


Overwriting code/nginx.conf


In [8]:
%%writefile code/conf.d/load-balancer.conf

# in the 'http' block
upstream nodejs {
    server node1:8081;
    server node2:8081;
}

server {
# in the 'server' block for HTTPS traffic
location /webapp/ {
    proxy_pass http://nodejs;
}
}


Overwriting code/conf.d/load-balancer.conf


In [ ]:
# %load code/app/server.js
var http = require('http');
var ip = require("ip");

// require the http module of node.js
var http = require('http');

// define the port of access for your server
const PORT = 8081;

// We need a function which handles requests and send response
function handleRequest(request, response){
    response.end('Server: '+ip.address()+' working properly. Requested URL : ' + request.url);
}

// Create a server
var myFirstServer = http.createServer(handleRequest);

// Start the server !
myFirstServer.listen(PORT, function(){
    // Callback triggered when server is successfully listening. Hurray!
    console.log("Server listening on: http://localhost:%s", PORT);
});


In [9]:
!docker-compose -f ./code/docker-compose.yml up

Removing code_ngix_1
Recreating 0e8614085d9c_0e8614085d9c_code_ngix_1 ... 
Recreating code_node1_1 ... 
Recreating 0e8614085d9c_0e8614085d9c_code_ngix_1
Recreating code_node2_1 ... 
Recreating code_node2_1
Recreating code_node1_1
Attaching to code_node1_1, code_ngix_1, code_node2_12mdone
node1_1  | 
node1_1  | > 2.-node@1.0.0 start /home/node/app
node1_1  | > node server.js
node1_1  | 
node1_1  | Server listening on: http://localhost:8081
node2_1  | 
node2_1  | > 2.-node@1.0.0 start /home/node/app
node2_1  | > node server.js
node2_1  | 
node2_1  | Server listening on: http://localhost:8081
ngix_1   | 172.24.0.1 - - [19/Nov/2018:10:33:05 +0000] "GET /webapp/ HTTP/1.1" 200 72 "http://localhost:8888/notebooks/LOAD_BALANCING_HTTPS/Load%20balancing%20and%20HTTPS.ipynb" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36" "-"
ngix_1   | 2018/11/19 10:33:08 [error] 6#6: *1 open() "/etc/nginx/html/favicon.ico" failed (2: No such file or di

# http://localhost:8080/webapp/

NOTE: if you reload the page the IP address changes showing that we are actually interacting with both the nodes. This is the proof of the load balancing.

# HTTPS in NGINX

In [8]:
!mkdir ./code/keys

mkdir: cannot create directory ‘./code/keys’: File exists


<img src='http://www.ia.nato.int/images/nia/pki/smime-01.jpg'>
<img src='https://www.tutorialspoint.com/cryptography/images/digital_certificate.jpg'>

Open a terminal in . and run 
```
openssl req -x509 -nodes -days 365 -newkey rsa:2048 -keyout ./code/keys/nginx-selfsigned.key -out ./code/keys/nginx-selfsigned.crt
```

This generates the couple of keys and the certificate

then 

```
openssl dhparam -out ./code/keys/dhparam.pem 2048
```

In [12]:
%%writefile code/docker-compose.yml

version: "2"

services:
  node1:
    image: "node:8"
    user: "node"
    working_dir: /home/node/app
    environment:
      - NODE_ENV=production
    volumes:
      - /home/andrea/Documents/University/Didattica/RC2018/notebooks/LOAD_BALANCING_HTTPS/code/app:/home/node/app
    expose:
      - 8081
    command: "npm start"
  node2:
    image: "node:8"
    user: "node"
    working_dir: /home/node/app
    environment:
      - NODE_ENV=production
    volumes:
      - /home/andrea/Documents/University/Didattica/RC2018/notebooks/LOAD_BALANCING_HTTPS/code/app:/home/node/app
    expose:
      - 8081
    command: "npm start"
  ngix:
    image: "nginx"
    volumes:
    
      - /home/andrea/Documents/University/Didattica/RC2018/notebooks/LOAD_BALANCING_HTTPS/code/var/www:/var/www
      - /home/andrea/Documents/University/Didattica/RC2018/notebooks/LOAD_BALANCING_HTTPS/code/nginx.conf:/etc/nginx/nginx.conf
      - /home/andrea/Documents/University/Didattica/RC2018/notebooks/LOAD_BALANCING_HTTPS/code/conf.d:/etc/nginx/conf.d
      - /home/andrea/Documents/University/Didattica/RC2018/notebooks/LOAD_BALANCING_HTTPS/code/keys:/etc/nginx/keys
    ports:
      - "8080:80"
      - "4443:443"
    expose:
      - 80
      - 443

Overwriting code/docker-compose.yml


In [13]:
%%writefile code/nginx.conf

user  nginx;
worker_processes  1;

error_log  /var/log/nginx/error.log warn;
pid        /var/run/nginx.pid;


events {
    worker_connections  1024;
}


http {

    ssl_certificate     /etc/nginx/keys/nginx-selfsigned.crt;
    ssl_certificate_key /etc/nginx/keys/nginx-selfsigned.key;
    ssl_ciphers         EECDH+AESGCM:EDH+AESGCM:AES256+EECDH:AES256+EDH;
    ssl_protocols       TLSv1.1 TLSv1.2;

    include       /etc/nginx/mime.types;
    default_type  application/octet-stream;

    log_format  main  '$remote_addr - $remote_user [$time_local] "$request" '
                      '$status $body_bytes_sent "$http_referer" '
                      '"$http_user_agent" "$http_x_forwarded_for"';

    access_log  /var/log/nginx/access.log  main;

    sendfile        on;
    #tcp_nopush     on;

    keepalive_timeout  65;

    #gzip  on;

    include /etc/nginx/conf.d/*.conf;
    #include /etc/nginx/conf.d/sites-available/*.conf;
    #include /etc/nginx/conf.d/sites-enabled/*.conf;
}


Overwriting code/nginx.conf


In [14]:
%%writefile code/conf.d/load-balancer.conf

# in the 'http' block
upstream nodejs {
    server node1:8081;
    server node2:8081;
}

server {
# in the 'server' block for HTTPS traffic

    listen              443 ssl default_server;
    listen              [::]:443 ssl default_server ;
    location /webapp/ {
        proxy_pass http://nodejs;
    }
}

Overwriting code/conf.d/load-balancer.conf


In [15]:
!docker-compose -f ./code/docker-compose.yml up

Recreating code_ngix_1 ... 
Recreating code_ngix_1
Starting code_node1_1 ... 
Starting code_node2_1 ... 
Starting code_node2_1
Starting code_node1_1
Attaching to code_node2_1, code_node1_1, code_ngix_1
node2_1  | 
node2_1  | > 2.-node@1.0.0 start /home/node/app
node2_1  | > node server.js
node2_1  | 
node2_1  | Server listening on: http://localhost:8081
node1_1  | 
node1_1  | > 2.-node@1.0.0 start /home/node/app
node1_1  | > node server.js
node1_1  | 
node1_1  | Server listening on: http://localhost:8081
ngix_1   | 172.24.0.1 - - [19/Nov/2018:10:42:24 +0000] "GET /webapp/ HTTP/1.1" 200 72 "http://localhost:8888/notebooks/LOAD_BALANCING_HTTPS/Load%20balancing%20and%20HTTPS.ipynb" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36" "-"
ngix_1   | 172.24.0.1 - - [19/Nov/2018:10:42:26 +0000] "GET /favicon.ico HTTP/1.1" 404 571 "https://localhost:4443/webapp/" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chro

# https://localhost:4443/webapp/

# The certificate is not recognised

<img src="assets/sec1.png">

# Indeed is not among the certificates of the certification authorities in the browser

<img src="assets/sec2.1.png">

# You can anyway proceed ... but you should know who issued the certificate 

<img src="assets/sec2.png">

# The evidence on load balancing: IP addresses alternate on reload

<img src="assets/sec3.png">
<img src="assets/sec4.png">